In [3]:
from keras import Input
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, concatenate
from keras.models import Model, load_model
import numpy as np

ar = np.load(r"E:\Trackmania Data\training_data_new\gesamt_processed.npy")
np.random.seed(1)
np.random.shuffle(ar)
# ar = ar[:100]
print("data length:", str(len(ar)))
x = ar[:, 0]
aux = ar[:, 1]
y = ar[:, 2]

C:\Users\mivog\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


data length: 8924


In [4]:
x = np.array([e for e in x])
x = x / 255


In [5]:
x.shape

(8924, 40, 150, 3)

In [6]:
aux = np.array([e for e in aux])
aux.shape

(8924, 16)

In [7]:
y = np.array([e for e in y])
y.shape

(8924, 4)

In [8]:
# Split data in training (70%), validation (30%)
x_train = x[:int(0.7 * len(x))]
x_val = x[int(0.7 * len(x)):]
# x_test = x[int(0.8 * len(x)):]
aux_train = aux[:int(0.7 * len(x))]
aux_val = aux[int(0.7 * len(x)):]
# aux_test = aux[int(0.8 * len(x)):]
y_train = y[:int(0.7 * len(x))]
y_val = y[int(0.7 * len(x)):]
# y_test = y[int(0.8 * len(x)):]


In [9]:
main_input = Input(shape=x_train[0].shape)
t = Conv2D(128, (5, 20), padding='same', activation='relu')(main_input)
t = MaxPooling2D(pool_size=(2, 8), strides=(2, 8))(t)
t = Conv2D(256, (5, 5), padding='same', activation='relu')(t)
t = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(t)
t = Dropout(0.4)(t)
t = Flatten()(t)
t = Dropout(0.4)(t)
convs_out = Dense(1024, activation='relu')(t)
auxiliary_output = Dense(4, activation='sigmoid', name='aux_output')(convs_out)

aux_input = Input(shape=aux_train[0].shape)
t = concatenate([convs_out, aux_input])
t = Dense(512, activation='relu')(t)
t = Dense(512, activation='relu')(t)

main_output = Dense(4, activation='sigmoid', name='main_output')(t)

model = Model(inputs=[main_input, aux_input], outputs=[main_output, auxiliary_output])
model.compile(optimizer='Adam', loss='binary_crossentropy', loss_weights=[1.0, 0.5])

In [10]:
# model = load_model(r"C:\Users\mivog\PycharmProjects\driving ai trackmania\weights\test_weights.38-0.05.hdf5")

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40, 150, 3)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 40, 150, 128) 38528       input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 20, 18, 128)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 20, 18, 256)  819456      max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

In [12]:
def get_class_weights(y):
    counter = [0, 0, 0, 0]
    for e in y:
        for i in range(4):
            if e[i] == 1:
                counter[i] += 1
    print("Zähler", counter)
    majority = max(counter)
    return {index: majority / counter[index] if counter[index] != 0 else 0 for index in range(4)}


In [20]:
name = "all_data_changed model"
epochs = 50
starting_epoch = 16

In [14]:
checkpoint = ModelCheckpoint("weights/" + name + "_weights.{epoch:02d}-{val_loss:.2f}.hdf5", period=2)
weights = get_class_weights(np.concatenate((y_train, y_val)))
weights[2] *= 10
weights[3] *= 10
print("Weights:", str(weights))

Zähler [8648, 0, 1172, 1172]
Weights: {0: 1.0, 1: 0, 2: 73.78839590443687, 3: 73.78839590443687}


In [15]:
tensboard = TensorBoard(log_dir="velo_logs/{}".format(name), histogram_freq=0, batch_size=32,
                        write_graph=False, write_grads=False, write_images=False, embeddings_freq=0,
                        embeddings_layer_names=None, embeddings_metadata=None
                        )


In [21]:
model.evaluate([x_train, aux_train], [y_train, y_train])

  32/6246 [..............................] - ETA: 3:26

  64/6246 [..............................] - ETA: 3:47

  96/6246 [..............................] - ETA: 3:40

 128/6246 [..............................] - ETA: 3:38

 160/6246 [..............................] - ETA: 3:46

KeyboardInterrupt: 

In [22]:
model.fit([x_train, aux_train], [y_train, y_train], epochs=epochs, validation_data=([x_val, aux_val], [y_val, y_val]),
          class_weight=weights, callbacks=[tensboard, checkpoint], batch_size=32, initial_epoch=starting_epoch)


Train on 6246 samples, validate on 2678 samples
Epoch 17/50


  32/6246 [..............................] - ETA: 11:55 - loss: 0.0458 - main_output_loss: 0.0311 - aux_output_loss: 0.0296

  64/6246 [..............................] - ETA: 11:29 - loss: 0.0261 - main_output_loss: 0.0175 - aux_output_loss: 0.0174

  96/6246 [..............................] - ETA: 11:16 - loss: 0.0453 - main_output_loss: 0.0300 - aux_output_loss: 0.0306

 128/6246 [..............................] - ETA: 10:54 - loss: 0.0383 - main_output_loss: 0.0251 - aux_output_loss: 0.0264

...

1248/6246 [====>.........................] - ETA: 9:02 - loss: 0.0348 - main_output_loss: 0.0231 - aux_output_loss: 0.0235

1280/6246 [=====>........................] - ETA: 8:58 - loss: 0.0357 - main_output_loss: 0.0238 - aux_output_loss: 0.0239

1312/6246 [=====>........................] - ETA: 8:54 - loss: 0.0355 - main_output_loss: 0.0237 - aux_output_loss: 0.0237

1344/6246 [=====>........................] - ETA: 8:51 - loss: 0.0351 - main_output_loss: 0.0234 - aux_output_loss: 0.0234

KeyboardInterrupt: 

In [23]:
model.save(name)